In [1]:
# IMPORTS

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tensorflow import keras
from tensorflow.keras.applications import vgg16
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# LOAD BASE MODEL
#IMG_SIZE = 224
#model = vgg16.VGG16(weights='imagenet', include_top=False , input_shape=(IMG_SIZE, IMG_SIZE, 3))

In [2]:
IMG_SIZE = 224
model = InceptionResNetV2(weights='imagenet', include_top=False , input_shape=(IMG_SIZE, IMG_SIZE, 3))

In [3]:
# PREPARE DATA

EAR_DIR = os.getcwd() + "/AWEDataset/awe/"
IMG_SIZE = 224
count = 0
image_data = []

for dir in os.listdir(EAR_DIR):
    if(dir != "other"):
        #print(dir)
        person_dir = EAR_DIR + dir + "/"
        for img_name in os.listdir(person_dir):
            if(img_name != "annotations.json"):
                #print("\t" + img)

                label = int(dir) - 1
                
                img_dir = person_dir + img_name
                
                img = cv2.imread(img_dir)
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_LINEAR)
                img = img/255;
                #print(img_dir, int(label))
                
                img_num = img_name[0:2]
                img_num = int(img_num)
                #print("\t" + str(img_num))
                
                img_consecutive_num = label*10 + img_num
                
                image_data.append((img, label, img_dir, img_consecutive_num))
                
                count+=1
      
                #res = model.predict(img_array)
                #print(res)
                #plt.imshow(img_array,cmap=plt.cm.binary)
                #plt.show()
       
    
print(count)  

#import random
#random.shuffle(image_data)

1000


In [4]:
# SPLIT DATA INTO TRAIN AND TEST

X_train = []
X_test = []
y_train = []
y_test = []

train_dirs = []
test_dirs = []


train_file_dir = os.getcwd() + "/AWEDataset/awe/other/train.txt"
test_file_dir = os.getcwd() + "/AWEDataset/awe/other/test.txt"

f = open(train_file_dir, "r")
train_imgs = list(map(int, f.read().rstrip().split(" ")))
#print(len(train_imgs))

f = open(test_file_dir, "r")
test_imgs = list(map(int, f.read().rstrip().split(" ")))
#print(len(test_imgs))

labels = set()


for img, label, img_dir, img_consecutive_num in image_data:
    #print(label, img_dir)
    labels.add(label)  
    #print(label, img_consecutive_num)
    
    if img_consecutive_num in train_imgs:
        X_train.append(img)
        y_train.append(label)
        train_dirs.append(img_dir)
        
  
    if img_consecutive_num in test_imgs:
        X_test.append(np.array(img))
        y_test.append(label)
        test_dirs.append(img_dir)
      
    
labels = list(labels)
labels.sort()
#print(len(labels))
#print(len(y_test))
#print(len(y_train))



X_train = np.array(X_train)
X_test = np.array(X_test)

print(X_train.shape)

y_train = np.array(y_train)
y_test = np.array(y_test)



(600, 224, 224, 3)


In [16]:
#print(model.summary())

model.trainable = False

base_inputs = model.layers[0].input
base_outputs = model.layers[-1].output

final_outputs = layers.GlobalAveragePooling2D()(base_outputs)

#final_outputs = layers.Dense(1024, activation="relu")(final_outputs)
final_outputs = layers.Dense(1024, activation="relu")(final_outputs)
#final_outputs = layers.Dense(512, activation="relu")(final_outputs)
#final_outputs = layers.Dense(2048, activation="relu")(final_outputs)
#final_outputs = layers.Dense(2048, activation="relu")(final_outputs)

#final_outputs = layers.Dense(512, activation="relu")(final_outputs)

final_outputs = layers.Dense(100, activation="softmax")(final_outputs)

new_model = keras.Model(inputs=base_inputs, outputs=final_outputs)

print(new_model.summary())

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [19]:

gen = ImageDataGenerator(rotation_range = 20, zoom_range = 0.10, 
fill_mode = "nearest", shear_range = 0.20, horizontal_flip = True, 
width_shift_range = 0.1, height_shift_range = 0.1)



new_model.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])


new_model.fit_generator(gen.flow(X_train, y_train, batch_size = 32), 
              epochs=20, shuffle=True, validation_data=(X_test, y_test))

Epoch 1/20
19/19 [==============================] - 46s 2s/step - loss: 0.5072 - accuracy: 0.8450 - val_loss: 4.4554 - val_accuracy: 0.2125
Epoch 2/20
19/19 [==============================] - 7s 378ms/step - loss: 0.4394 - accuracy: 0.9033 - val_loss: 4.4058 - val_accuracy: 0.2525
Epoch 3/20
19/19 [==============================] - 7s 381ms/step - loss: 0.3771 - accuracy: 0.9083 - val_loss: 4.4254 - val_accuracy: 0.2225
Epoch 4/20
19/19 [==============================] - 7s 380ms/step - loss: 0.3388 - accuracy: 0.9300 - val_loss: 4.5918 - val_accuracy: 0.2350
Epoch 5/20
19/19 [==============================] - 7s 382ms/step - loss: 0.3760 - accuracy: 0.8983 - val_loss: 4.4665 - val_accuracy: 0.2275
Epoch 6/20
19/19 [==============================] - 7s 381ms/step - loss: 0.3543 - accuracy: 0.9233 - val_loss: 4.4533 - val_accuracy: 0.2225
Epoch 7/20
19/19 [==============================] - 7s 383ms/step - loss: 0.2964 - accuracy: 0.9183 - val_loss: 4.3527 - val_accuracy: 0.2325
Epoch 8/

In [ ]:
#new_model.save('augmented_ear_50_epochs_inception_1024_3.model')

In [11]:

model_predictions = []

for i in range(len(X_test)):
    img = X_test[i]
    label = y_test[i]
    img_dir=test_dirs[i]
    img_expand = img.reshape(1, IMG_SIZE, IMG_SIZE, 3)
    
    res = new_model.predict(img_expand)
    predictions = res[0]
    #print(len(predictions))
    
    entry = {
        #"img": img,
        "label": label,
        "img_dir": img_dir,
        "predictions": predictions
    }
    
    model_predictions.append(entry)

results = new_model.evaluate(X_test, y_test)
print(results)

model_eval = {
    
    "model_predictions": model_predictions,
    "results": results
}

import pickle
import os


aug_or_no = "aug" # "aug" "no_aug"
model_base = "InceptionResNetV2" # "InceptionResNetV2" "vgg16"
epochs = "80_epochs" # "80_epochs" "50_epochs" "30_epochs"


dir_to_save = os.getcwd() + "/results/" + aug_or_no + "/" + model_base + "/" + epochs + "/"


pickle_out = open(dir_to_save + "model_eval.pickle","wb")
pickle.dump(model_eval, pickle_out)
pickle_out.close()

print("done")


13/13 [==============================] - 3s 216ms/step - loss: 4.8555 - accuracy: 0.2375
[4.855477333068848, 0.23749999701976776]
done
